## Clasificación de Textos con Word Embeddings

<img src="figs/fig-diagrama-clasificador2.png" width="900">

# Entrenamiento del clasificador

# 1. Cargar el conjunto de datos

In [2]:
import pandas as pd
import fasttext

dataset = pd.read_json("./data/data_emotions_es.json", lines=True)


# 2. Cargar modelo de vectores pre-entrenados de Word Embeddings: FastText

In [3]:

# Descargar el modelo para el español de la página de FastText

# cambiar la ruta del modelo preentrenado
ft = fasttext.load_model('./data/cc.es.300.bin')


# 3. Transformar el texto de entrenamiento en un vector denso: 300 dimensiones definido por FastText

In [4]:

dataset["text_embed"] = dataset["text"].map(lambda x: ft.get_sentence_vector(x))


In [ ]:
dataset.head()

# 4. Crear la matriz de vectores y etiquetas para el entrenamiento

In [6]:
import numpy as np

# Cada renglón representa un documento codificado en un texto de embeddings
X = np.vstack(dataset['text_embed'].to_numpy())
Y = dataset['klass'].to_numpy()

In [ ]:
print(X.shape)
print(Y.shape)


## 5. Preparar los conjuntos de datos  (datasets) para entrenamiento y para probar el rendimiento del clasificador

In [8]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)


X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)


In [ ]:
X_train.shape, Y_train

# 6. Entrenar al clasificador

### Importamos LinearSVC del paquete sklearn

In [ ]:
from sklearn.svm import LinearSVC

clasificador_svc = LinearSVC(random_state=42)

# Entrenamos al clasificador

clasificador_svc.fit(X_train, Y_train)


### Predicción de los datos del conjuntos de prueba con el modelo entrenado

In [11]:
y_pred = clasificador_svc.predict(X_test)


### Inspección de los resultados de los primeros N ejemplos de prueba

In [ ]:
print("textos: ")
print(dataset.text.loc[:5])
print("clase esperada: ",Y_test[:5])
print("clase predicha: ", y_pred[:5])

### Mostrar la predicción de la clase original 

In [ ]:
# Obten las primeras N predicciones

print("textos: ")
print(dataset.text.loc[:5])
print("clase esperada: ",le.inverse_transform(Y_test[:5]))
print("clase predicha: ", le.inverse_transform(y_pred[:5]))


# 7. Evaluando el desempeño

## Métricas de Evaluación
 - #### Las métricas precisión, recall y F1 son fundamentales para evaluar el rendimiento de un clasificador




$$ Accuracy = \frac{total~ TP + total~TN}{total~muestras} $$

$$ Precision_c = \frac{ TP_c}{TP_c + FP_c} $$

$$ Recall_c = \frac{ TP_c}{TP_c + FN_c} $$

$$ F1-score_c= 2 \times \frac{ Precision_c \times Recall_c}{Precision_c + Recall_c} $$

$$ macro-F1-score= \frac{ 1 }{|Clases|} \sum{F1-score_c} $$

## Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(Y_test, y_pred)

In [ ]:
# para la clase 0, la precisión es la siguiente
tp= 77
fp = 28+17+21
tp/(tp + fp)

## Métricas

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print("P=", precision_score(Y_test, y_pred, average='macro'))
print("R=", recall_score(Y_test, y_pred, average='macro'))
print("F1=", f1_score(Y_test, y_pred, average='macro'))
print("Acc=", accuracy_score(Y_test, y_pred))


## Inspección del desempeño por clase

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred, digits=4, zero_division='warn'))

## 8. Guadar el modelo para despliegue de la aplicación

### Se recomienda generar un módulo para el despliegue del modelo (Por ejemplo clasificadorembeddingsTODO.py)

In [ ]:
from clasificadorembeddingsTODO import ClasificadorEmbeddignsTODO
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_json("./data/data_emotions_es.json", lines=True)

# Descargar el modelo para el español de la página de FastText
ft = fasttext.load_model('/Volumes/data/temp/cc.es.300.bin')

# Crear los vectores de embeddings
dataset["text_embed"] = dataset["text"].map(lambda x: ft.get_sentence_vector(x))

# Cada renglón representa un documento codificado en un texto de embeddings
X = np.vstack(dataset['text_embed'].to_numpy())
Y = dataset['klass'].to_numpy()


# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)


X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)


clasificador_svc = ClasificadorEmbeddingsTODO(embeddings_model=ft)
clasificador_svc.setLabelEncoder(le)
clasificador_svc.fit(X_train, Y_train)

# guardar el modelo de clasificación
with open('./modelos/modelo_embeddings_svc.pkl','wb') as file:
    pickle.dump(clasificador_svc, file)
    
print(clasificador_svc.predict([X_test[0]]))